In [17]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt

from shallow_water import PeriodicLinearShallowWater
from plotting import plot_wind_arrows

In [8]:
import warnings
warnings.simplefilter(action='ignore', category=DeprecationWarning)

In [67]:
def run_linear_model(beta=2.28e-11, Rd=1e6, matgill_yexp=2):
    """
    Params:
    
     - Radius of deformation: Rd = sqrt(2 c / beta)
     - beta
    """
    nx = 256
    ny = nx//2 + 1


    # Radius of deformation: Rd = sqrt(2 c / beta)
    #Rd = 1000.0e3  # Fix Rd at 1000km

    Lx = 10*Rd
    Ly = 5*Rd

    #beta=2.28e-11
    c = Rd**2 * beta  # Kelvin wave speed: c = sqrt(gH)
    g = 1.0

    H = c**2/g       # Set phi baseline from deformation radius

    cfl = 0.7         # For numerical stability CFL = |u| dt / dx < 1.0
    dx  = Ly / nx
    dt = np.floor(cfl * dx / (c*4))

    tau = 500000
    nu = 1000

    atmos = PeriodicLinearShallowWater(nx, ny, Lx, Ly, beta=beta, f0=0.0, g=g, H=H, dt=dt, nu=nu)

    x, y = np.meshgrid(atmos.phix/Rd, atmos.phiy/Rd)
    k = np.pi/2
    Q0 = H * 0.01
    Q = (Q0*np.exp(-(1/2)*y**matgill_yexp)*np.cos(k*x))
    Q[np.abs(x) > 1] = 0
    Q = Q.T

    @atmos.add_forcing
    def matsuno_gill(model):    
        u, v, h = model.state
        du, dv, dh = np.zeros_like(model.state)

        # forcing terms for the linear matsuno gill problem
        du = - u/tau
        dv = - v/tau
        dh = (Q - h)/tau

        return np.array([du, dv, dh])


    N = int(tau/dt*3)
    for i in range(N):
        atmos.step()
        if i%int(N*0.1)==0:
            print(i)
    
    da_forcing = xr.DataArray(Q.T, dims=['y', 'x'],
                      coords={'x': atm.phix[:,0]/Rd,
                              'y': atm.phiy[0,:]/Rd})
    
    da_height = xr.DataArray(atmos.h.T, dims=['y', 'x'],
                      coords={'x': atm.phix[:,0]/Rd,
                              'y': atm.phiy[0,:]/Rd})
    
    return da_height, da_forcing


In [ ]:
%%time
height, forcing = run_linear_model()

/home/users/andreww/miniconda3/envs/plumeworld/lib/python3.7/site-packages/ipykernel_launcher.py:50: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


0
1006
2012
3018
4024
5030
6036


In [ ]:
fig, ax = plt.subplots(dpi=100, figsize=(6, 4))

(height-height.mean('y')).plot(ax=ax)

forcing.plot.contour(ax=ax, levels=[1], colors='grey', linewidths=0.8)

plot_wind_arrows(atmos, (x,y), narrows=(15,15), hide_below=0.05, color='k')
ax.set_aspect('equal')
ax.set_xlabel('x  (L$_{d}$)')
ax.set_ylabel('y  (L$_{d}$)')

ax.set_title('State at T=%.2f days' % (atmos.t / 86400.0))

#plt.colorbar()

In [ ]:
%%time

height_halfRd, forcing_halfRd = run_linear_model(Rd=5e5)

In [ ]:
fig, ax = plt.subplots(dpi=100, figsize=(6, 4))

(height_halfRd-height_halfRd.mean('y')).plot(ax=ax)

forcing_halfRd.plot.contour(ax=ax, levels=[1], colors='grey', linewidths=0.8)

plot_wind_arrows(atmos, (x,y), narrows=(15,15), hide_below=0.05, color='k')
ax.set_aspect('equal')
ax.set_xlabel('x  (L$_{d}$)')
ax.set_ylabel('y  (L$_{d}$)')

ax.set_title('State at T=%.2f days' % (atmos.t / 86400.0))

#plt.colorbar()